# Load Back Projected ICA Data

In [1]:
import scipy.io as sio

In [10]:
filepath = '/Users/apple/Desktop/eeglab14_1_2b/ICA_Data/EEGData1.mat'
mat_contents = sio.loadmat(filepath)
ica = mat_contents['data']
temp_trial = ica[:,:,1]
trial = ica.shape[2]
print("{}: {}".format("Total number of trials is", trial))
channel = temp_trial.shape[0]
print("{}: {}".format("Total number of channels in each trial is", channel))
timepoint = temp_trial.shape[1]
print("{}: {}".format("Total number of time points in per channel per trial is", timepoint))

Total number of trials is: 40
Total number of channels in each trial is: 32
Total number of time points in per channel per trial is: 8064


# Testing between 1 trial 2 channels with GC

In [13]:
import statsmodels.tsa.stattools as stm
import numpy as np

In [14]:
#we just gonna pick temp_trial
print(temp_trial.shape)

(32, 8064)


In [36]:
a = np.asarray(temp_trial[0,:])
b = np.asarray(temp_trial[1,:])
x = np.vstack((a, b)).T
print(x.shape)

(8064, 2)


In [57]:
from nolitsa import dimension # for FNN
maxlag = 5 #dimension.fnn(x[0])

In [58]:
addconst = True
verbose = True

In [59]:
result = stm.grangercausalitytests(x, maxlag, addconst, verbose)
optimal_lag = -1
F_test = -1.0
for key in result.keys():
    _F_test_ = result[key][0]['params_ftest'][0]
    if _F_test_ > F_test:
        F_test = _F_test_
        optimal_lag = key


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=137.4970, p=0.0000  , df_denom=8060, df_num=1
ssr based chi2 test:   chi2=137.5481, p=0.0000  , df=1
likelihood ratio test: chi2=136.3881, p=0.0000  , df=1
parameter F test:         F=137.4970, p=0.0000  , df_denom=8060, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=311.0022, p=0.0000  , df_denom=8057, df_num=2
ssr based chi2 test:   chi2=622.3905, p=0.0000  , df=2
likelihood ratio test: chi2=599.5351, p=0.0000  , df=2
parameter F test:         F=311.0022, p=0.0000  , df_denom=8057, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=283.9087, p=0.0000  , df_denom=8054, df_num=3
ssr based chi2 test:   chi2=852.4664, p=0.0000  , df=3
likelihood ratio test: chi2=810.3369, p=0.0000  , df=3
parameter F test:         F=283.9087, p=0.0000  , df_denom=8054, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=215.8010, p=0.

In [63]:
print("{} {}".format("We are going to look into the GC with Optimal Lag of", optimal_lag))

We are going to look into the GC with Optimal Lag of 2


We consider the p-value of the test as a measure for Granger causality: rejection of ℋ0 (p < 0.03) signifies Granger causality, acceptance means non-causality.

In [65]:
from statsmodels.tsa.ar_model import AR

The causality relations drawn from systems with very small values of |det(ΛˆI)| are not meaningful